In [ ]:
from networkx.algorithms import approximation
import math
import networkx as nx
import random
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import shutil
import csv
import glob
import ast
import numpy as np
get_ipython().run_line_magic('matplotlib', 'inline')
from networkx import assortativity
from networkx import distance_measures

In [ ]:
# functions for visualizing graphs (from correlation matrices) and calculating small world measures
# functions written by Dirk Gütlin, https://digyt.github.io/graph_measures_implementation/

def remove_self_connections(matrix):
    """Removes the self-connections of a network graph."""
    for i in range(len(matrix)):
        matrix[i, i] = 0
    return matrix


def invert_matrix(matrix):
    """Invert a matrix from 0 to 1, dependent on whether dealing with distances or weight strengths."""
    new_matrix = 1 - matrix.copy()
    return remove_self_connections(new_matrix)


def plot_graph(graph_matrix, title="Connectivity/Graph Matrix", show=True, cb=True):
    plt.pcolormesh(graph_matrix, vmin=0, vmax=1)
    plt.title(title)
    plt.ylabel('Channels/Nodes')
    plt.xlabel('Channels/Nodes')
    if cb:
        plt.colorbar(ticks=[0, .5, 1])
    if show:
        plt.show()

def weighted_shortest_path(matrix):
    """
    Calculate the shortest path lengths between all nodes in a weighted graph.

    This is an implementation of the Floyd-Warshall algorithm for finding the shortest
    path lengths of an entire graph matrix. Implementation taken from:
    https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm

    This implementation is actually identical to scipy.sparse.csgraph.floyd_warshall,
    which we found after the implementation.
    """
    matrix = invert_matrix(matrix)

    n_nodes = len(matrix)
    distances = np.empty([n_nodes, n_nodes])
    for i in range(n_nodes):
        for j in range(n_nodes):
            distances[i, j] = matrix[i, j]

    for i in range(n_nodes):
        distances[i, i] = 0

    for k in range(n_nodes):
        for i in range(n_nodes):
            for j in range(n_nodes):
                if distances[i, j] > distances[i, k] + distances[k, j]:
                    distances[i, j] = distances[i, k] + distances[k, j]

    return distances


def weighted_characteristic_path_length(matrix):
    """Calculate the characteristic path length for weighted graphs."""
    n_nodes = len(matrix)
    min_distances = weighted_shortest_path(matrix)

    sum_vector = np.empty(n_nodes)
    for i in range(n_nodes):
        # calculate the inner sum
        sum_vector[i] = (1/(n_nodes-1)) * np.sum([min_distances[i, j]
                                                  for j in range(n_nodes) if j != i])

    return (1/n_nodes) * np.sum(sum_vector)


def weighted_node_degree(matrix):
    """Calculate the node degree for all nodes in a weighted graph."""
    return np.sum(matrix, axis=-1)


def unweighted_node_degree(matrix):
    """Calculate the node degree for all nodes in a weighted graph."""
    return np.sum(np.ceil(matrix), axis=-1)


def weighted_triangle_number(matrix):
    """Calculate the weighted geometric mean of triangles around i for all nodes i in a weighted graph."""
    n_nodes = len(matrix)

    mean_vector = np.empty([n_nodes])
    for i in range(n_nodes):
        triangles = np.array([[matrix[i, j] * matrix[i, k] * matrix[j, k]
                             for j in range(n_nodes)] for k in range(n_nodes)])**(1/3)
        mean_vector[i] = (1/2) * np.sum(triangles, axis=(0, 1))

    return mean_vector


def weighted_clustering_coeff(matrix):
    """Calculate the clustering coefficient for a weighted graph."""
    n = len(matrix)
    t = weighted_triangle_number(matrix)
    # here we use the !max possible weights as reference
    k = unweighted_node_degree(matrix)
    return (1/n) * np.sum((2 * t)/(k * (k - 1)))


def weighted_clustering_coeff_z(matrix):
    """Zhang's CC is an alternative clustering coefficient which should work better for our case See Samaräki et al. (2008)."""
    n_nodes = len(matrix)
    ccs = []
    for i in range(n_nodes):
        upper = np.sum([[matrix[i, j] * matrix[j, k] * matrix[i, k]
                       for k in range(n_nodes)] for j in range(n_nodes)])
        lower = np.sum([[matrix[i, j] * matrix[i, k]
                       for k in range(n_nodes) if j != k] for j in range(n_nodes)])
        ccs.append(upper/lower)

    return np.mean(ccs)

def lattice_reference(G, niter=1, D=None, seed=np.random.seed(np.random.randint(0, 2**30))):
    """Latticize the given graph by swapping edges. Works similar to networkx' lattice reference."""
    from networkx.utils import cumulative_distribution, discrete_sequence

    # Instead of choosing uniformly at random from a generated edge list,
    # this algorithm chooses nonuniformly from the set of nodes with
    # probability weighted by degree.
    G = G.copy()
    keys = [i for i in range(len(G))]
    degrees = weighted_node_degree(G)
    cdf = cumulative_distribution(degrees)  # cdf of degree

    nnodes = len(G)
    nedges = nnodes * (nnodes - 1) // 2  # NOTE: assuming full connectivity
    if D is None:
        D = np.zeros((nnodes, nnodes))
        un = np.arange(1, nnodes)
        um = np.arange(nnodes - 1, 0, -1)
        u = np.append((0,), np.where(un < um, un, um))

        for v in range(int(np.ceil(nnodes / 2))):
            D[nnodes - v - 1, :] = np.append(u[v + 1:], u[: v + 1])
            D[v, :] = D[nnodes - v - 1, :][::-1]

    niter = niter * nedges
    ntries = int(nnodes * nedges / (nnodes * (nnodes - 1) / 2))
    swapcount = 0

    for i in range(niter):
        n = 0
        while n < ntries:
            # pick two random edges without creating edge list
            # choose source node indices from discrete distribution
            (ai, bi, ci, di) = discrete_sequence(
                4, cdistribution=cdf, seed=seed)
            if len(set((ai, bi, ci, di))) < 4:
                continue  # picked same node twice
            a = keys[ai]  # convert index to label
            b = keys[bi]
            c = keys[ci]
            d = keys[di]

            is_closer = D[ai, bi] >= D[ci, di]
            is_larger = (G[ai, bi] >= G[ci, di])
            if is_closer and is_larger:
                # only swap if we get closer to the diagonal

                ab = G[a, b]
                cd = G[c, d]
                G[a, b] = cd
                G[b, a] = cd
                G[c, d] = ab
                G[d, c] = ab

                swapcount += 1
                break
            n += 1
    return G


def random_reference(G, niter=1, D=None, seed=np.random.seed(np.random.randint(0, 2**30))):
    """Latticize the given graph by swapping edges. Works similar to networkx' random reference."""
    from networkx.utils import cumulative_distribution, discrete_sequence

    # Instead of choosing uniformly at random from a generated edge list,
    # this algorithm chooses nonuniformly from the set of nodes with
    # probability weighted by degree.
    G = G.copy()
    keys = [i for i in range(len(G))]
    degrees = weighted_node_degree(G)
    cdf = cumulative_distribution(degrees)  # cdf of degree

    nnodes = len(G)
    nedges = nnodes * (nnodes - 1) // 2  # NOTE: assuming full connectivity
    if D is None:
        D = np.zeros((nnodes, nnodes))
        un = np.arange(1, nnodes)
        um = np.arange(nnodes - 1, 0, -1)
        u = np.append((0,), np.where(un < um, un, um))

        for v in range(int(np.ceil(nnodes / 2))):
            D[nnodes - v - 1, :] = np.append(u[v + 1:], u[: v + 1])
            D[v, :] = D[nnodes - v - 1, :][::-1]

    niter = niter * nedges
    ntries = int(nnodes * nedges / (nnodes * (nnodes - 1) / 2))
    swapcount = 0

    for i in range(niter):
        n = 0
        while n < ntries:
            # pick two random edges without creating edge list
            # choose source node indices from discrete distribution
            (ai, bi, ci, di) = discrete_sequence(
                4, cdistribution=cdf, seed=seed)
            if len(set((ai, bi, ci, di))) < 4:
                continue  # picked same node twice
            a = keys[ai]  # convert index to label
            b = keys[bi]
            c = keys[ci]
            d = keys[di]

            # only swap if we get closer to the diagonal

            ab = G[a, b]
            cd = G[c, d]
            G[a, b] = cd
            G[b, a] = cd
            G[c, d] = ab
            G[d, c] = ab

            swapcount += 1
            break

    return G

def random_reference_nx(matrix, niter=10):
    G = nx.convert_matrix.from_numpy_array(matrix)
    G_ref = nx.algorithms.smallworld.random_reference(G, niter=niter)
    return nx.convert_matrix.to_numpy_array(G_ref)


def lattice_reference_nx(matrix, niter=10):
    G = nx.convert_matrix.from_numpy_array(matrix)
    G_ref = nx.algorithms.smallworld.lattice_reference(G, niter=niter)
    return nx.convert_matrix.to_numpy_array(G_ref)


def weighted_sw_sigma(matrix, n_avg=1):
    """Calculate the weighted small world coefficient sigma of a matrix."""
    sigmas = []
    for i in range(n_avg):
        random_graph = random_reference(matrix)
        C = weighted_clustering_coeff_z(matrix)
        C_rand = weighted_clustering_coeff_z(random_graph)
        L = weighted_characteristic_path_length(matrix)
        L_rand = weighted_characteristic_path_length(random_graph)
        sigma = (C/C_rand) / (L/L_rand)
        sigmas.append(sigma)

    return np.mean(sigmas)


def weighted_sw_omega(matrix, n_avg=1):
    """Calculate the weighted small world coefficient omega of a matrix."""
    omegas = []
    for i in range(n_avg):
        random_graph = random_reference(matrix)
        lattice_graph = lattice_reference(matrix)
        C = weighted_clustering_coeff_z(matrix)
        C_latt = weighted_clustering_coeff_z(lattice_graph)
        L = weighted_characteristic_path_length(matrix)
        L_rand = weighted_characteristic_path_length(random_graph)
        omega = (L_rand/L) / (C/C_latt)
        omegas.append(omega)

    return np.mean(omegas)


def weighted_sw_index(matrix, n_avg=1):
    """Calculate the weighted small world coefficient omega of a matrix."""
    indices = []
    for i in range(n_avg):
        random_graph = random_reference(matrix)
        lattice_graph = lattice_reference(matrix)
        C = weighted_clustering_coeff_z(matrix)
        C_rand = weighted_clustering_coeff_z(random_graph)
        C_latt = weighted_clustering_coeff_z(lattice_graph)
        L = weighted_characteristic_path_length(matrix)
        L_rand = weighted_characteristic_path_length(random_graph)
        L_latt = weighted_characteristic_path_length(lattice_graph)
        index = ((L - L_latt) / (L_rand - L_latt)) * \
            ((C - C_rand) / (C_latt - C_rand))
        indices.append(index)
    return np.mean(indices)

In [ ]:
# calculate weighted graph measures from correlation matrices (each matrix corresponding to a word, MNI x MNI)
PATH='~/corr_matrices'
# Create a dataframe that will store the graph measures for all CSVs
graph_df_abs=pd.DataFrame(columns=['weighted_characteristic_path_length','weighted_clustering_coeff_z',
                                                                 'weighted_sw_sigma','average_clustering','diameter','radius',
                                                                 'transitivity','degree_assortativity_coefficient','global_efficiency',
                                                                 'local_efficiency'])
for condition in os.listdir(PATH):
    for csv in os.listdir(PATH+'/'+condition):
        #print(condition,csv)
        data = pd.read_csv(PATH+'/'+condition+'/'+csv)
        data.set_index('Unnamed: 0',inplace=True)
        # drop empty rows
        data = data.dropna(how='all')
        # drop empty columns
        data = data.dropna(axis=1, how='all')
        # Fill empty values with zeros
        data.fillna(0, inplace=True)
        # Create weighted graph - convert to absolute values
        matrix = data.abs().values
        name=condition+'_'+csv
        graph_df_abs.loc[name,'weighted_characteristic_path_length']=weighted_characteristic_path_length(matrix)
        graph_df_abs.loc[name,'weighted_clustering_coeff_z']=weighted_clustering_coeff_z(matrix)
        graph_df_abs.loc[name,'weighted_sw_sigma']=weighted_sw_sigma(matrix)
        # Create distance graph before applying built-in networkx functions
        G_gt=nx.from_numpy_array(1-matrix)
        # Remove self loops in the graph
        G_gt.remove_edges_from(nx.selfloop_edges(G_gt))
        graph_df_abs.loc[name,'average_clustering']=nx.average_clustering(G_gt,weight='weight')
        graph_df_abs.loc[name,'diameter']=nx.diameter(G_gt,weight='weight')
        graph_df_abs.loc[name,'radius']=nx.radius(G_gt,weight='weight')
        graph_df_abs.loc[name,'transitivity']=nx.transitivity(G_gt)
        graph_df_abs.loc[name,'degree_assortativity_coefficient']=nx.degree_assortativity_coefficient(G_gt,weight='weight')
        graph_df_abs.loc[name,'global_efficiency']=nx.global_efficiency(G_gt)
        graph_df_abs.loc[name,'local_efficiency']=nx.local_efficiency(G_gt)

# save the dataframe as a csv
graph_df_abs.to_csv('~/graph_measures.csv')